<a href="https://colab.research.google.com/github/lawalAfeez820/NLP/blob/main/hotel_sentiment_analysis_with_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOTEL REVIEW SENTIMENT ANALYSIS WITH LSTM.

### Sentiment analysis is one of the test clasificaion problem in NLP,Sentiment analysis tool is used to detect and understand customers/users feelings. Sentiment analysis tools generate insights into how companies can enhance the customer experience and improve customer service.

### Businesses can use machine learning based sentiment analysis software to examine the speech and text for positive and negative sentiment about their brand.

#### I developed a simple machine learning model for an hotel review sentiment using LSTM with keras framework. The output of the model is shown below

---



In [1]:
import tensorflow as tf
import pandas as pd
import re

import string
import spacy
nlp=spacy.load("en_core_web_sm")
import gensim
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!pip install contractions
import contractions

import nltk
nltk.download('stopwords')
stopword=list(stopwords.words("english"))
stopword.remove("no")
stopword.remove("not")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
def fix_contraction(sentence):
    sentence=sentence.lower()
    return contractions.fix(sentence)
def preprocessing(sentence):
  
    stopword=set(stopwords.words("english"))
    stopword.discard("no")
    stopword.discard("not")
    pattern=re.compile("[^A-Za-z ]")

    sentence=sentence.lower()
    sentence=fix_contraction(sentence)
    sentence=pattern.sub(" ",sentence)
    
    

    Word_in_sentence=word_tokenize(sentence)
    join_sentence=[list(nlp(word).sents)[0].lemma_ for word in Word_in_sentence if word not in stopword]
   
    joined_sentence=" ".join(join_sentence)
    return joined_sentence

In [4]:
# loading the dataset into pandas dataframe

data=pd.read_csv("/content/description.csv" ,delimiter="," )

data.head()

,Description,Is_Response,fixed contractions
0,book arlington night read review trip advisor ...,not happy,we booked at the arlington for - nights after ...
1,historic beautifully appoint hotel one block m...,happy,a very historic beautifully appointed hotel on...
2,hotel clean update relatively new room furnitu...,not happy,if this hotel were cleaner and updated with re...
3,hotel locate minute walk alamo hotel pretty go...,happy,this hotel is located about - minute walk to t...
4,spend night great hotel staff helpful breakfas...,happy,we spent - nights at this great hotel - the st...


In [5]:
data["Is_Response"].value_counts(normalize=True)

not happy    0.5
happy        0.5
Name: Is_Response, dtype: float64

In [6]:
data["Is_Response"].value_counts()

not happy    4000
happy        4000
Name: Is_Response, dtype: int64

In [7]:
data=data[["Description","Is_Response"]]
data.head()

,Description,Is_Response
0,book arlington night read review trip advisor ...,not happy
1,historic beautifully appoint hotel one block m...,happy
2,hotel clean update relatively new room furnitu...,not happy
3,hotel locate minute walk alamo hotel pretty go...,happy
4,spend night great hotel staff helpful breakfas...,happy


In [8]:
data["Is_Response"]=data["Is_Response"].map({"happy":1,"not happy":0})

In [9]:
average_word_per_review=sum([len(i.split())for i in data["Description"]])/len(data["Description"])
average_word_per_review

83.503

In [10]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y=train_test_split(data["Description"],data["Is_Response"],stratify=data["Is_Response"],random_state=2,test_size=0.1)

In [11]:
type(train_y)

pandas.core.series.Series

In [12]:
x_train_data=tf.data.Dataset.from_tensor_slices(train_x)
y_train_data=tf.data.Dataset.from_tensor_slices(train_y)

x_test_data=tf.data.Dataset.from_tensor_slices(test_x)
y_test_data=tf.data.Dataset.from_tensor_slices(test_y)

In [13]:
#preprocessing the dataset with keras preprocessing layer

#Preprocessing the text using preprocessing layer text vectorization

import re,string
def custom_standadization(text):

  lowercase=tf.strings.lower(text)

  stripped_html=tf.strings.regex_replace(lowercase,"<br />" ," ")

  stripped_number=tf.strings.regex_replace(stripped_html,"[\d-]"," ")

  stripped_punc=tf.strings.regex_replace(stripped_number,"[%s]" % re.escape(string.punctuation),"")

  return stripped_punc

In [14]:
from pandas.compat.numpy.function import TAKE_DEFAULTS
from keras.layers import TextVectorization

vectorize=TextVectorization(
    standardize=custom_standadization,
    output_sequence_length=int(average_word_per_review)
)

In [15]:
y_train_data.element_spec

TensorSpec(shape=(), dtype=tf.int64, name=None)

In [16]:
y_train_data=y_train_data.map(lambda x: tf.reshape(x,[1]))
y_trest_data=y_test_data.map(lambda x: tf.reshape(x,[1]))

In [17]:
y_train_data.element_spec

TensorSpec(shape=(1,), dtype=tf.int64, name=None)

In [18]:
vectorize.adapt(x_train_data)

In [19]:
len(vectorize.get_vocabulary())

16398

In [20]:
x_train_data=x_train_data.map(lambda x : vectorize(x))
x_test_data=x_test_data.map(lambda x : vectorize(x))

In [21]:
for i in x_train_data.take(2):
  print(i)


tf.Tensor(
[  54    2  131   48   50   59 2376   52   62 2030  200   97   12 1424
  247   65   82  322 1560 1202   18  176   17    3   85   52 1252   62
  152  952  108 1514  287  370  135   24   82  462 1127   21    4  113
   24  531 1076 7137 3020   24  374  110   14  559    3   41  176   14
  190   86   33   41   17   59   79   76  268  132  116 1097 1717  165
   40 1066  515   16  617  393  284    2   40 7011 2598 1063   19], shape=(83,), dtype=int64)
tf.Tensor(
[   3  176 1159   19  277  158  199 1991  417 1354  253  124  136  168
   27  770 1946   50 2232  712  636 6597 2948 1433  561  796  672 1590
  803 1024 1973 2533  247  117   13   11  192  126   30 2533  155  511
    4   60  235   40  193  556 3303    3  117   47 1227 1127  390  135
 2755 3489  165 1876  473    6    4  121  245   15 3067    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0], shape=(83,), dtype=int64)


In [22]:
train_ds=tf.data.Dataset.zip((x_train_data,y_train_data))
test_ds=tf.data.Dataset.zip((x_test_data,y_test_data))
train_ds.element_spec

(TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 TensorSpec(shape=(1,), dtype=tf.int64, name=None))

In [23]:

def shape(x,y):
  x.set_shape([83])
  y=y

  return x,y

In [24]:
train_ds=train_ds.map(shape)
test_ds=test_ds.map(shape)

In [25]:
train_ds.element_spec

(TensorSpec(shape=(83,), dtype=tf.int64, name=None),
 TensorSpec(shape=(1,), dtype=tf.int64, name=None))

In [26]:

train_ds=train_ds.shuffle(23).padded_batch(128,drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

test_ds=test_ds.shuffle(23).padded_batch(128,drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

In [27]:
train_ds.element_spec

(TensorSpec(shape=(128, 83), dtype=tf.int64, name=None),
 TensorSpec(shape=(128, 1), dtype=tf.int64, name=None))

In [28]:
from keras.layers import LSTM,Input,Embedding,Dropout,Dense,Flatten
from keras.models import Model

input=Input(shape=(83))
embedding=Embedding(len(vectorize.get_vocabulary()),50)(input)
dropout1=Dropout(0.4)(embedding)
lstm1=LSTM(128,return_sequences=True)(dropout1)
dropout2=Dropout(0.4)(lstm1)
lstm2=LSTM(64,return_sequences=True)(dropout2)
dropout3=Dropout(0.3)(lstm2)
flatten=Flatten()(dropout3)
output=Dense(1,activation="sigmoid")(flatten)

model=Model(input,output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 83)]              0         
                                                                 
 embedding (Embedding)       (None, 83, 50)            819900    
                                                                 
 dropout (Dropout)           (None, 83, 50)            0         
                                                                 
 lstm (LSTM)                 (None, 83, 128)           91648     
                                                                 
 dropout_1 (Dropout)         (None, 83, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 83, 64)            49408     
                                                                 
 dropout_2 (Dropout)         (None, 83, 64)            0     

In [29]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [30]:

from keras.callbacks import EarlyStopping
es=EarlyStopping(patience=5,mode="min")

history=model.fit(train_ds,validation_data=test_ds,epochs=30,callbacks=[es])

Epoch 1/30
56/56 [==============================] - 11s 48ms/step - loss: 0.5503 - accuracy: 0.7063 - val_loss: 0.4090 - val_accuracy: 0.8307
Epoch 2/30
56/56 [==============================] - 1s 16ms/step - loss: 0.3155 - accuracy: 0.8669 - val_loss: 0.3907 - val_accuracy: 0.8346
Epoch 3/30
56/56 [==============================] - 1s 14ms/step - loss: 0.2329 - accuracy: 0.9096 - val_loss: 0.4650 - val_accuracy: 0.8307
Epoch 4/30
56/56 [==============================] - 1s 15ms/step - loss: 0.1782 - accuracy: 0.9304 - val_loss: 0.4958 - val_accuracy: 0.8216
Epoch 5/30
56/56 [==============================] - 1s 14ms/step - loss: 0.1443 - accuracy: 0.9482 - val_loss: 0.4859 - val_accuracy: 0.8099
Epoch 6/30
56/56 [==============================] - 1s 14ms/step - loss: 0.1305 - accuracy: 0.9487 - val_loss: 0.6234 - val_accuracy: 0.8047
Epoch 7/30
56/56 [==============================] - 1s 14ms/step - loss: 0.0925 - accuracy: 0.9653 - val_loss: 0.6427 - val_accuracy: 0.8177


In [31]:
val_loss,val_accuracy=model.evaluate(test_ds)

6/6 [==============================] - 0s 6ms/step - loss: 0.6427 - accuracy: 0.8177


In [32]:
prediction=model.predict(test_ds)

In [33]:
prediction=(prediction.flatten()>0.5).astype(int)
prediction

array([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,

In [46]:
model.predict(vectorize("interest").numpy().reshape(1,83))[0][0]>0.4

True

In [79]:
def analyse_review(review):

  review=preprocessing(review)

  prediction=model.predict(vectorize(review).numpy().reshape(1,83))[0][0]>=0.5

  if prediction :

    return "POSITIVE REVIEW"

  else:

    return "NEGATIVE REVIEW"



# SAMPLES

In [80]:
analyse_review("""I was told that my room would have several amenities like microwave and a.c. unfortunately There was only a mini fridge 
                 in my room and the air conditioner didn’t work at all which is and was a problem in 100⁰ weather.""")

'NEGATIVE REVIEW'

In [81]:
analyse_review("This hotel use to be great,I'm not sure if they switched owners but it has gone down hill from food to service.")

'NEGATIVE REVIEW'

In [82]:
analyse_review("""The room was clean,very comfortable and the staff was amazing.They went over and beyond to help make our stay enjoyable.
                 I highly recommend this hotel for anyone visiting downtown""")

'POSITIVE REVIEW'

In [83]:
analyse_review("""They were extremely accommodating and allowed us to check in early at like 10am. We got to hotel super early and I didn’t wanna wait.
                 So this was a big plus. The sevice was exceptional as well. Would definitely send a friend there.""")

'POSITIVE REVIEW'

In [84]:
analyse_review("""The rooms stink. They are not clean. I requested a non smoking room and both rooms smelled like smoke. Bathrooms were gross and bugs were everywhere! ! 
                 The door did not seem secure and was not evened out so bugs got in easily. The second room was full of gnats.""")

'NEGATIVE REVIEW'